In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sqlite3

In [2]:
fintech_research_sql = sqlite3.connect(database="../data/fintech_research.sqlite")

crsp_monthly = (pd.read_sql_query(
    sql=("SELECT permno, gvkey, month, ret_excess, mktcap, " 
         "mktcap_lag, exchange FROM crsp_monthly"),
    con=fintech_research_sql,
    parse_dates={"month"})
  .dropna()
)

book_equity = (pd.read_sql_query(
    sql="SELECT gvkey, datadate, be FROM compustat",
    con=fintech_research_sql, 
    parse_dates={"datadate"})
  .dropna()
  .assign(
    month=lambda x: (
      pd.to_datetime(x["datadate"]).dt.to_period("M").dt.to_timestamp()
    )
  )
)

In [3]:
me = (crsp_monthly
  .assign(sorting_date=lambda x: x["month"]+pd.DateOffset(months=1))
  .rename(columns={"mktcap": "me"})
  .get(["permno", "sorting_date", "me"])
)

bm = (book_equity
  .merge(crsp_monthly, how="inner", on=["gvkey", "month"])
  .assign(bm=lambda x: x["be"]/x["mktcap"],
          sorting_date=lambda x: x["month"]+pd.DateOffset(months=6))
  .assign(comp_date=lambda x: x["sorting_date"])
  .get(["permno", "gvkey", "sorting_date", "comp_date", "bm"])
)

data_for_sorts = (crsp_monthly
  .merge(bm, 
         how="left", 
         left_on=["permno", "gvkey", "month"], 
         right_on=["permno", "gvkey", "sorting_date"])
  .merge(me, 
         how="left", 
         left_on=["permno", "month"], 
         right_on=["permno", "sorting_date"])
  .get(["permno", "gvkey", "month", "ret_excess", 
        "mktcap_lag", "me", "bm", "exchange", "comp_date"])
)

data_for_sorts = (data_for_sorts
  .sort_values(by=["permno", "gvkey", "month"])
  .groupby(["permno", "gvkey"])
  .apply(lambda x: x.assign(
      bm=x["bm"].fillna(method="ffill"), 
      comp_date=x["comp_date"].fillna(method="ffill")
    )
  )
  .reset_index(drop=True)
  .assign(threshold_date = lambda x: (x["month"]-pd.DateOffset(months=12)))
  .query("comp_date > threshold_date")
  .drop(columns=["comp_date", "threshold_date"])
  .dropna()
)

/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/4024672676.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  bm=x["bm"].fillna(method="ffill"),
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/4024672676.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  comp_date=x["comp_date"].fillna(method="ffill")
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/4024672676.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(


In [4]:
def assign_portfolio(data, exchanges, sorting_variable, n_portfolios):
    """Assign portfolio for a given sorting variable."""
    
    breakpoints = (data
      .query(f"exchange in {exchanges}")
      .get(sorting_variable)
      .quantile(np.linspace(0, 1, num=n_portfolios+1), 
                interpolation="linear")
      .drop_duplicates()
    )
    breakpoints.iloc[0] = -np.Inf
    breakpoints.iloc[breakpoints.size-1] = np.Inf
    
    assigned_portfolios = pd.cut(
      data[sorting_variable],
      bins=breakpoints,
      labels=range(1, breakpoints.size),
      include_lowest=True,
      right=False
    )
    
    return assigned_portfolios

In [5]:
## independent sort

value_portfolios = (data_for_sorts
  .groupby("month")
  .apply(lambda x: x.assign(
      portfolio_bm=assign_portfolio(
        data=x, sorting_variable="bm", n_portfolios=5, exchanges=["NYSE"]
      ),
      portfolio_me=assign_portfolio(
        data=x, sorting_variable="me", n_portfolios=5, exchanges=["NYSE"]
      )
    )
  )
  .reset_index(drop=True)
  .groupby(["month", "portfolio_bm", "portfolio_me"])
  .apply(lambda x: pd.Series({
      "ret": np.average(x["ret_excess"], weights=x["mktcap_lag"])
    })
  )
  .reset_index()
)

value_portfolios

/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/1939614639.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/1939614639.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["month", "portfolio_bm", "portfolio_me"])
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/1939614639.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is 

,month,portfolio_bm,portfolio_me,ret
0,1970-08-01,1,1,0.056839
1,1970-08-01,1,2,NaN
2,1970-08-01,1,3,0.137413
3,1970-08-01,1,4,NaN
4,1970-08-01,1,5,-0.005300
...,...,...,...,...
15720,2022-12-01,5,1,-0.041637
15721,2022-12-01,5,2,-0.053698
15722,2022-12-01,5,3,-0.070272
15723,2022-12-01,5,4,-0.076946


In [6]:
value_premium = (value_portfolios
  .groupby(["month", "portfolio_bm"])
  .aggregate({"ret": "mean"})
  .reset_index()
  .groupby("month")
  .apply(lambda x: pd.Series({
    "value_premium": (
        x.loc[x["portfolio_bm"] == x["portfolio_bm"].max(), "ret"].mean() - 
          x.loc[x["portfolio_bm"] == x["portfolio_bm"].min(), "ret"].mean()
      )
    })
  )
  .aggregate({"value_premium": "mean"})
)

value_premium

/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/4252429700.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["month", "portfolio_bm"])
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/4252429700.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


value_premium    0.00382
dtype: float64

In [7]:
## dependent sort
value_portfolios = (data_for_sorts
  .groupby("month")
  .apply(lambda x: x.assign(
      portfolio_me=assign_portfolio(
        data=x, sorting_variable="me", n_portfolios=5, exchanges=["NYSE"]
      )
    )
  )
  .reset_index(drop=True)
  .groupby(["month", "portfolio_me"])
  .apply(lambda x: x.assign(
      portfolio_bm=assign_portfolio(
        data=x, sorting_variable="bm", n_portfolios=5, exchanges=["NYSE"]
      )
    )
  )
  .reset_index(drop=True)
  .groupby(["month", "portfolio_bm", "portfolio_me"])
  .apply(lambda x: pd.Series({
      "ret": np.average(x["ret_excess"], weights=x["mktcap_lag"])
    })
  )
  .reset_index()
)

value_premium = (value_portfolios
  .groupby(["month", "portfolio_bm"])
  .aggregate({"ret": "mean"})
  .reset_index()
  .groupby("month")
  .apply(lambda x: pd.Series({
    "value_premium": (
        x.loc[x["portfolio_bm"] == x["portfolio_bm"].max(), "ret"].mean() -
          x.loc[x["portfolio_bm"] == x["portfolio_bm"].min(), "ret"].mean()
      )
    })
  )
  .aggregate({"value_premium": "mean"})
)

value_premium

/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/341501395.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/341501395.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["month", "portfolio_me"])
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_37452/341501395.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in 

value_premium    0.003289
dtype: float64